In [ ]:
!pip install transformers datasets
!pip install xformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
# import time
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from datasets import load_dataset
# import torch
# import time

# # Load MRPC dataset
# task = "mrpc"
# dataset = load_dataset("glue", task)

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
# model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)
# model.eval()  # Set model to evaluation mode

# # Prepare example input (take a sample from the dataset)
# sentence1 = dataset["test"][13]["sentence1"]
# sentence2 = dataset["test"][13]["sentence2"]

# # Tokenize input text
# inputs = tokenizer(sentence1, sentence2, return_tensors="pt", truncation=True, padding=True, max_length=512)

# # Measure Inference Latency
# def measure_latency(model, inputs, iterations=100):
#     # Warm-up runs to make sure the model is ready
#     for _ in range(10):
#         with torch.no_grad():
#             _ = model(**inputs)

#     # Measure latency
#     torch.cuda.synchronize()
#     start_time = time.perf_counter()
#     for _ in range(iterations):
#         with torch.no_grad():
#             outputs = model(**inputs)
#             logits = outputs.logits
#     torch.cuda.synchronize()

#     end_time = time.perf_counter()

#     avg_latency = (end_time - start_time) / iterations
#     print(f"Average Inference Latency: {avg_latency:.6f} seconds")

# # Run latency measurement
# measure_latency(model, inputs)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Inference Latency: 0.015304 seconds


In [ ]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import evaluate

# Load MRPC dataset
task = "mrpc"
dataset = load_dataset("glue", task)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)
model.eval()  # Set model to evaluation mode

# Validation Accuracy Code (Separate)
def measure_validation_accuracy(model, validation_dataset):
    accuracy_metric = evaluate.load("accuracy")
    all_predictions = []
    all_labels = []

    for example in validation_dataset:
        inputs = tokenizer(
            example["sentence1"],
            example["sentence2"],
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        labels = example["label"]

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()

        all_predictions.extend(predictions)
        all_labels.append(labels)

    # Compute accuracy
    accuracy = accuracy_metric.compute(predictions=all_predictions, references=all_labels)
    print(f"Validation Accuracy: {accuracy['accuracy']:.4f}")

# Inference Latency Code
def measure_latency(model, inputs, iterations=100):
    # Warm-up runs to make sure the model is ready
    for _ in range(10):
        with torch.no_grad():
            _ = model(**inputs)

    # Measure latency
    torch.cuda.synchronize()
    start_time = time.perf_counter()
    for _ in range(iterations):
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
    torch.cuda.synchronize()

    end_time = time.perf_counter()

    avg_latency = (end_time - start_time) / iterations
    print(f"Average Inference Latency: {avg_latency:.6f} seconds")

# Measure validation accuracy
validation_dataset = dataset["validation"]
measure_validation_accuracy(model, validation_dataset)

# Measure inference latency on a sample example
sentence1 = dataset["test"][13]["sentence1"]
sentence2 = dataset["test"][13]["sentence2"]
inputs = tokenizer(sentence1, sentence2, return_tensors="pt", truncation=True, padding=True, max_length=512)
measure_latency(model, inputs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Accuracy: 0.4387
Average Inference Latency: 0.014255 seconds
